## Thử với dữ liệu mẫu

In [2]:
import json
from collections import Counter
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import pandas as pd
import os

In [3]:
# ⚠️ Thay đường dẫn tới file của bạn
file_path = "../mental_health_eng_viet/mental_health_eng_viet.csv"  # Hoặc .xlsx nếu cần

# Đọc file
df = pd.read_csv(file_path)  # hoặc pd.read_excel(file_path)

# Drop dòng thiếu dữ liệu (nếu có)
df = df.dropna(subset=["question", "answer"])

In [5]:
df.head(5)

,question,answer
0,Tôi nghĩ nó có thể là sự kết hợp của nhiều thứ...,Cảm nhận được sức nặng của những kỳ vọng bên n...
1,"I've been trying to, but it's been challenging...",Letting go can be incredibly difficult. It's o...
2,Không phải lúc này. Sự hướng dẫn của bạn ngày ...,Tôi rất vui vì có thể giúp được anh. Hãy nhớ r...
3,Gần đây tôi cảm thấy thực sự buồn. Thật khó để...,", tôi có thể cảm nhận được nỗi buồn của bạn. L..."
4,"Vâng, tôn giáo là một tôn giáo lớn. Tôi đã chọ...",Điều đó hẳn là vô cùng khó khăn. Việc họ không...


In [6]:
# Chuyển từng cặp thành message format
converted_data = []
for _, row in df.iterrows():
    conversation = {
        "messages": [
            {"role": "user", "content": str(row['question'])},
            {"role": "assistant", "content": str(row['answer'])}
        ]
    }
    converted_data.append(conversation)


In [8]:

# Tạo thư mục output
output_dir = "chatgpt_format"
os.makedirs(output_dir, exist_ok=True)

# Chia nhỏ file (50,000 hội thoại mỗi file)
chunk_size = 50000
for i in range(0, len(converted_data), chunk_size):
    chunk = converted_data[i:i + chunk_size]
    with open(f"{output_dir}/conversations_part{i//chunk_size + 1}.json", "w", encoding="utf-8") as f:
        json.dump(chunk, f, ensure_ascii=False, indent=2)

print(f"✅ Đã chia nhỏ thành {len(os.listdir(output_dir))} file tại {output_dir}")


✅ Đã chia nhỏ thành 20 file tại chatgpt_format


In [ ]:
# Thống kê độ dài câu hỏi/trả lời
question_lengths = []
answer_lengths = []
word_counter = Counter()

for conv in df:
    if "messages" not in conv or len(conv["messages"]) < 2:
        continue  # bỏ qua bản ghi không đủ cấu trúc

    question = conv["messages"][0]["content"]
    answer = conv["messages"][1]["content"]

    question_lengths.append(len(question.split()))
    answer_lengths.append(len(answer.split()))

    word_counter.update(re.findall(r"\w+", question.lower()))
    word_counter.update(re.findall(r"\w+", answer.lower()))


NameError: name 'data' is not defined

In [ ]:
print(f"Độ dài trung bình câu hỏi: {sum(question_lengths)/len(question_lengths):.2f} từ")
print(f"Độ dài trung bình câu trả lời: {sum(answer_lengths)/len(answer_lengths):.2f} từ")
print(f"Từ vựng duy nhất: {len(word_counter)}")

In [ ]:
# WordCloud từ vựng
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counter)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud từ vựng toàn bộ hội thoại")
plt.show()

In [ ]:
# Biểu đồ độ dài câu hỏi/trả lời
sns.histplot(question_lengths, bins=30, kde=True, color='blue', label='Câu hỏi')
sns.histplot(answer_lengths, bins=30, kde=True, color='green', label='Câu trả lời')
plt.title("Phân phối độ dài câu")
plt.xlabel("Số từ")
plt.legend()
plt.show()